# Catboost

LLD 10/22/2019

## Introduction

https://tech.yandex.com/catboost/
    
CatBoost is an open-source gradient boosting on decision trees library developed by Yandex researchers and engineers. It's the successor of the MatrixNet algorithm that is widely used within the industry for ranking tasks, forecasting and making recommendations. 

It's Accurate, Robust (reduces the need for extensive hyper-parameter tuning), Easy-to-use (integrated with scikitlearn), practical (uses categorical features directly and scalably) and **extensible (allows specifying custom loss functions)**

Do not use OHE which reduces prediction accuracy 

- CatBoostClassifier
- CatBoostRegressor

https://towardsdatascience.com/https-medium-com-talperetz24-mastering-the-new-generation-of-gradient-boosting-db04062a7ea2

## Algorithm - Classic Gradient Boosting

<img src="classic_gradient_boosting.png">

<img src="gradient_tree.png">

## Catboost Secret Sauce

Catboost introduces two critical algorithmic advances - the implementation of **ordered boosting**, a permutation-driven alternative to the classic algorithm, and an innovative algorithm for **processing categorical features**.

### Categorical Feature Handling - Ordered Target Statistic 

Applying target statistic carelessly can result in target leakage. To fight this prediction shift CatBoost uses a more effective strategy. It relies on the ordering principle and is inspired by online learning algorithms which get training examples sequentially in time. In this setting, the values of TS for each example rely only on the observed history.
To adapt this idea to a standard offline setting, Catboost introduces an artificial “time”— a random permutation σ1 of the training examples.
Then, for each example, it uses all the available “history” to compute its Target Statistic.
Note that, using only one random permutation, results in preceding examples with higher variance in Target Statistic than subsequent ones. To this end, CatBoost uses different permutations for different steps of gradient boosting.

### Categorical Feature Handling - One Hot Encoding

Catboost uses a one-hot encoding for all the features with at most one_hot_max_size unique values. The default value is 2.

<img src="catboost_one_hot.png">

### Orederd Boosting

CatBoost has two modes for choosing the tree structure, Ordered and Plain. **Plain mode** corresponds to a combination of the standard GBDT algorithm with an ordered Target Statistic. <br>
In **Ordered mode** boosting we perform a random permutation of the training examples - σ2, and maintain n different supporting models - M1, . . . , Mn such that the model Mi is trained using only the first i samples in the permutation.
At each step, in order to obtain the residual for j-th sample, we use the model Mj−1.
Unfortunately, this algorithm is not feasible in most practical tasks due to the need of maintaining n different models, which increase the complexity and memory requirements by n times. Catboost implements a modification of this algorithm, on the basis of the gradient boosting algorithm, using one tree structure shared by all the models to be built.

<img src="ordered_boosting.png">

In [ ]:
# read catboost_playground 

### POOL

In [ ]:
# Catboost way of preparing dataset for model
class Pool(data, 
           label=None,
           cat_features=None,
           column_description=None,
           pairs=None,
           delimiter='\t',
           has_header=False,
           weight=None, 
           group_id=None,
           group_weight=None,
           subgroup_id=None,
           pairs_weight=None
           baseline=None,
           feature_names=None,
           thread_count=-1)

### CatBoostClassifer

In [ ]:
import catboost
from catboost import CatBoostClassifier, Pool, cv 
from sklearn.model_selection import train_test_split

### Parameters

https://catboost.ai/docs/concepts/python-reference_parameters-list.html 
- **loss_function**: 'Logloss' for binomial classification, 'MultiClass' for more than 2 categories, 'RMSE' for regression 
- **iterations**: Num of trees to build
- **depth**: Depth of the tree
- **learning_rate**: default 0.03
- **l2_leaf_reg**: Coefficient at the L2 regularization term of the cost function
- **class_weights**: The value used as multiplier for the object weights. For imbalanced dataset classification the weight multiplier can be set to 1 for class 0 and (sum_negatives/sum_positives) for class 1. For instance, target rate of 2% then class_weights=[1,49] (result in bad performance when training on APost model with ConsumerView + OT Features)
- **boosting_type**: Boosting scheme. 
    Possible values:
        Ordered — Usually provides better quality on small datasets, but it may be slower than the Plain scheme. <br>
        Plain — The classic gradient boosting scheme.

- **border_count**: The number of splits for numerical features
- **subsample**: Sample rate for bagging, default 0.66
- **bagging_temperature**: This parameter is responsible for Bayesian bootstrap.
Bayesian bootstrap is used by default in classification and regression modes. In ranking we use Bernoulli bootstrap by default.In bayesian bootstrap each object is assigned random weight. If bagging temperature is equal to 1 then weights are sampled from exponential distribution. If bagging temperature is equal to 0 then all weights are equal to 1. By changing this parameter from 0 to +infty you can controll intensity of the bootstrap.
- **bootstrap_type**: Defines the method for sampling the weights of objects (Poisson, Bayesian, Bernoulli)
- **random_strength**: This parameter helps to overcome overfitting of the model. When selecting a new split, each possible split gets a score (for example, by how much does adding this split improve the loss function on train). After that all scores are sorted and a split with the highest score is selected.
The scores are not random. This parameter adds a normally distributed random variable to the score of the feature. It has zero mean and variance that is larger in the start of the training and decreases during the training. random_strength is the multiplier of the variance.
- **min_data_in_leaf**: The minimum number of training samples in a leaf (support by GPU only)
- **max_leaves**: Max # of trees in the resulting tree 


- **custom_metric**: metric values to output during training. These functions are not optimized and are displayed for informational purposes only. For classification, most used are Logloss, CrossEntropy, Precision, Recall, F1, Accuracy, and 'RMSE'/'MAE'/'MSLE' for regression https://catboost.ai/docs/concepts/loss-functions.html
- **eval_metric** is for overfitting detection and best model selection. A user-defined function can also be set as the value. 
- **use_best_model**: True if a validation set is input
- **nan_mode**: method for processing missing values ('Forbidden', 'Min' and 'Max')
- **one_hot_max_size**: Use one-hot encoding for all categorical features with a number of different values less than or equal to the given parameter value.
- **random_seed**



In [ ]:
from skopt.space import Real, Integer
catboost_params_space = [Real(1e-7, 1, prior='log-uniform', name='learning_rate'), 
                Integer(2, 10, name='max_depth'),
                Real(0.5, 1.0, name='subsample'),
                Real(0.5, 1.0, name='colsample_bylevel'),  
                Integer(1, 10, name='gradient_iterations'), 
                Real(1.0, 16.0, name='scale_pos_weight'), 
                Real(0.0, 1.0, name='bagging_temperature'), 
                Integer(1, 20, name='random_strength'), 
                Integer(2, 25, name='one_hot_max_size'),
                Real(1.0, 100, name='reg_lambda')]

### Baseline Catboost

In [ ]:
df['is_train'] = np.random.uniform(0, 1,len(df)) <= .5
train, val = df[df['is_train']==True], df[df['is_train']==False]

In [ ]:
# updated 03/05/2019

def run_catboost(feature_set, 
                 iterations, 
                 depth, 
                 learning_rate, 
                 l2_leaf_reg, 
                 bagging_temperature, 
                 border_count, 
                 df, predict_var):
    # Create two new dataframes, one with the training rows, one with the test rows
#     train, test = df[df['is_train']==True], df[df['is_train']==False]
    # Show the number of observations for the test and training dataframes
    print('Number of observations in the training data:', len(train))
    print('Number of observations in the development data:', len(val))
    print('Number of observations in the test data:',len(test))

    X = train[feature_set]
    y = train[target1].values

    # CatBoost validates estimations against a subset of your training set
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234)  

    categorical_feature_indices =np.where(X_train.dtypes==np.dtype('object'))[0]

    model=CatBoostClassifier(loss_function='Logloss',
                             iterations=iterations, 
                             depth=depth, 
                             learning_rate=learning_rate,
                             l2_leaf_reg=l2_leaf_reg,
#                              subsample=0.66, (cause error)
                             bagging_temperature = bagging_temperature, 
                             border_count=border_count, 
                             random_strength=1,
                             metric_period=50,
                             od_type='Iter',
                             od_wait=50,                           
                             custom_metric=['Accuracy'],
#                              custom_loss=['Accuracy'],
                             eval_metric='AUC',
#                              eval_metric=['Logloss','Accuracy'],
                             use_best_model=True,
                             thread_count=32,
                             random_seed=42)

    model.fit(X_train, y_train, cat_features=categorical_feature_indices, eval_set=(X_test, y_test), 
#               verbose=True,
              plot=True)
    
    from sklearn.metrics import confusion_matrix
    print(confusion_matrix(y_true = y_test, y_pred = model.predict(X_test), labels=(0,1)))

#     df[predict_var]  = model.predict_proba(df[feature_set])[:,1]
    return model

In [ ]:
def charting(predict_var, df, feature_set, model):
    train, validate, test = df[df['true_split']=='train'],df[df['true_split']=='val'],df[df['true_split']=='test']
    
    train.loc[:,predict_var]  = model.predict_proba(train[feature_set])[:,1]
    validate.loc[:,predict_var]  = model.predict_proba(validate[feature_set])[:,1]
    test.loc[:,predict_var]  = model.predict_proba(test[feature_set])[:,1]

#     train = train.sort_values(predict_var, ascending=False)
#     validate = validate.sort_values(predict_var, ascending=False)
#     test = test.sort_values(predict_var, ascending=False)

#     train.loc[:,'ntile']=range(len(train))
#     validate.loc[:,'ntile']=range(len(validate))
#     test.loc[:,'ntile']=range(len(test))

    train.loc[:,'ntile']=pd.qcut(train['ntile'], 100, labels=False)
    validate.loc[:,'ntile']=pd.qcut(validate['ntile'], 100, labels=False)
    test.loc[:,'ntile']=pd.qcut(test['ntile'], 100, labels=False)
    
    june_train = train.groupby('ntile').agg({target1:['count','sum','mean']}).reset_index()
    june_train.columns=['ntile','count','sum','avg_cr_train']

    june_validate = validate.groupby('ntile').agg({target1:['count','sum','mean']}).reset_index()
    june_validate.columns=['ntile','count','sum','avg_cr_val']

    june_test = test.groupby('ntile').agg({target1:['count','sum','mean']}).reset_index()
    june_test.columns=['ntile','count','sum','avg_cr_test']
    
    train_val_test = pd.concat([june_train, june_validate['avg_cr_val'], june_test['avg_cr_test']], axis=1)
    
    print(train_val_test)
    train_val_test[['avg_cr_train','avg_cr_val','avg_cr_test']].plot(figsize=(10,8))
    
    return train_val_test 

In [ ]:
# two group
def charting(predict_var, df, feature_set, model):
    train, validate = df2[df2['is_train']==True], df2[df2['is_train']==False]
    
    train.loc[:,predict_var]  = model.predict_proba(train[feature_set])[:,1]
    validate.loc[:,predict_var]  = model.predict_proba(validate[feature_set])[:,1]

    train = train.sort_values(predict_var, ascending=False)
    validate = validate.sort_values(predict_var, ascending=False)

    train.loc[:,'ntile']=range(len(train))
    validate.loc[:,'ntile']=range(len(validate))

    train.loc[:,'ntile']=pd.qcut(train['ntile'], 100, labels=False)
    validate.loc[:,'ntile']=pd.qcut(validate['ntile'], 100, labels=False)
    
    june_train = train.groupby('ntile').agg({target1:['count','sum','mean']}).reset_index()
    june_train.columns=['ntile','count','sum','avg_cr_train']

    june_validate = validate.groupby('ntile').agg({target1:['count','sum','mean']}).reset_index()
    june_validate.columns=['ntile','count','sum','avg_cr_val']
  
    train_val_test = pd.concat([june_train, june_validate['avg_cr_val']], axis=1)
    
    print(train_val_test)
    train_val_test[['avg_cr_train','avg_cr_val']].plot(figsize=(10,8))
    
    return train_val_test 

In [ ]:
iterations=1500
depth=6
learning_rate=0.03
l2_leaf_reg=80
border_count=10
bagging_temperature=10

model1, df = run_catboost(all_features, 
                          iterations, 
                          depth, 
                          learning_rate, 
                          l2_leaf_reg, 
                          bagging_temperature, 
                          border_count, 
                          df, 
                          'sale_hat')

In [ ]:
print(model1.get_best_score())

In [ ]:
pd.options.display.float_format='{:,.2f}'.format
feature_importances = list(zip(df[all_features], model1.feature_importances_))
feature_importances = pd.DataFrame(feature_importances, columns=['feature_name', 'importance'])
feature_importances.set_index('feature_name').sort_values(by='importance',ascending=False)
# feature_importances.set_index('feature_name').sort_values(by='importance',ascending=True).plot(kind='barh', color='blue', alpha=0.5, fontsize=10, figsize=(10,8))

In [ ]:
train_val_test = charting('sale_hat', df2, all_features, model1)

In [ ]:
top30 = list(feature_importances.sort_values(by='importance',ascending=False)['feature_name'].head(30))

### CV

In [ ]:
cv_data = cv(model.get_params(),
             Pool(X_train,y_train,cat_features=categorical_feature_indices),
             fold_count=3,
             plot=True)

In [ ]:
print('the best cv accuracy is :{}'.format(np.max(cv_data["b'Accuracy'_test_avg"])))

In [ ]:
from sklearn.metrics import accuracy_score
print('the test accuracy is :{:.6f}'.format(accuracy_score(y_test, model.predict(X_test))))

### Using Object Weights

In [ ]:
df['MaxRPS'].fillna(0, inplace=True)
df['weight']=np.log(df['MaxRPS']+1)
df['weight2']=df['MaxRPS'].apply(lambda x:1.5 if x>50 else 1)

In [ ]:
train=df[df['is_train']==True]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le=LabelEncoder()

for i in ['MarketingChannel',
 'mobile_browser',
 'brand_name',
 'PaymentMethod',
 'device_os',
 'DeviceGroup',
 'ReasonForCheckingCredit',
 'IsMobileBrowser',
 'PoliticalPersona',
 'Mosaic Household',
 'State Abbreviation',
 'Mosaic ZIP4',
 'IsDesktopBrowser',
 'SRVY:HH Lifestyl:Buying:Sports Related',
 'IsTablet',
 'MobileUsers',
 'EducationModel',
 'City Name',
 'Estimated Household Income Range Code V6',
 'Mosaic Global ZIP4',
 'ZIP Locality',
 'Household Composition',
 'SRVY:HH Lifestyl:Credit Cards:Other CardPremium']:
    df[i]=le.fit_transform(df[i].astype(str))
    df[i]=df[i].astype(str)
    
X = train[top50]
y = train['Sale'].values

    # CatBoost validates estimations against a subset of your training set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234)  

categorical_feature_indices =np.where(X_train.dtypes==np.dtype('object'))[0]

import catboost as cat
from catboost import Pool
train_set = cat.Pool(data=X_train, label=y_train, cat_features=categorical_feature_indices, weight=df.loc[X_train.index,'weight2'])
test_set  = cat.Pool(data=X_test, label=y_test, cat_features=categorical_feature_indices, weight=df.loc[X_test.index,'weight2'])

In [ ]:
model5=CatBoostClassifier(loss_function='Logloss',
                             iterations=2000, 
                             depth=6, 
                             learning_rate=0.03,
                             l2_leaf_reg=80,
#                              subsample=0.66,
                             bagging_temperature = 10,
                             border_count=10, 
                             random_strength=1,
                             metric_period=50,
                             od_type='Iter',
                             od_wait=50,                           
#                              custom_metric=['Accuracy'],
# #                              custom_loss=['Accuracy'],
#                              eval_metric='AUC',
                             use_best_model=True,
                             thread_count=32,
                             random_seed=42)

model5.fit(train_set, eval_set=test_set, 
#            verbose=True,
           plot=True
          )

### Catboost & Bayesian Search

categorical features need to convert into numeric format and set them to object 

In [ ]:
X=train[cr_top_30]
y=train[target1]

In [ ]:
X.isnull().sum() # no NA in cat features

In [ ]:
cat_features_short = [i for i in X.columns if X[i].dtypes==np.dtype('object')]
num_features_short = [i for i in X.columns if X[i].dtypes!=np.dtype('object')]
print(cat_features_short)
print(num_features_short)

In [ ]:
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder 

le=LabelEncoder()

for i in cat_features_short:
    X[i+'_le'] = le.fit_transform(X[i].astype(str))

In [ ]:
# convert it back to object
for i in ['MarketingChannel_le', 'DeviceType_le', 'ScoreRange_le', 'DeviceOS_le',
       'LastReasonForCheckingCredit_le', 'WorstPayStatus_le',
       'BrowserFamily_le']:
    X[i]=X[i].astype(str)

In [ ]:
cr_top30_le = num_features_short + ['MarketingChannel_le', 'DeviceType_le', 'ScoreRange_le', 'DeviceOS_le',
       'LastReasonForCheckingCredit_le', 'WorstPayStatus_le',
       'BrowserFamily_le']

print(cr_top30_le)
print(len(cr_top30_le))

In [ ]:
X2 = train[cr_top30_le]
y = train[target]

# CatBoost validates estimations against a subset of your training set
X_train, X_test, y_train, y_test = train_test_split(X2, y, random_state=1234)  

categorical_feature_indices =np.where(X_train.dtypes==np.dtype('object'))[0]

In [ ]:
train_ris=Pool(X_train,label=y_train,cat_features=categorical_feature_indices)
test_ris=Pool(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score
def catcv(iterations, depth, l2_leaf_reg, bagging_temperature,border_count):
    params={
            'iterations':int(iterations),
            'depth':int(depth),
            'l2_leaf_reg':l2_leaf_reg,
            'bagging_temperature':bagging_temperature,
            'border_count':int(border_count)
        }
    params['loss_function']='Logloss'
    params['eval_metric']='AUC'
    val = cv(
             train_ris,
             params,
             iterations=5
            ).mean()
    return val['test-AUC-mean']

bayesian= {
            'iterations':(200,1000),
            # max depth is 16
            'depth':(1,16),
            'l2_leaf_reg':(1,100),
            'bagging_temperature':(0,100),
            'border_count':(1,255)
           }

In [ ]:
from bayes_opt import BayesianOptimization

bayesian_search =BayesianOptimization(catcv,bayesian)

bayesian_search.maximize(n_iter=3)

In [ ]:
bayesian_search.res['max']['max_params']

In [ ]:
bayesian_params=bayesian_search.res['max']['max_params']

In [ ]:
bayesian_params['loss_function']='Logloss'
bayesian_params['eval_metric']='AUC'

In [ ]:
final_params={
            'iterations':int(bayesian_params['iterations']),
            'depth':int(bayesian_params['depth']),
            'l2_leaf_reg':bayesian_params['l2_leaf_reg'],
            'bagging_temperature':bayesian_params['bagging_temperature'],
            'border_count':int(bayesian_params['border_count'])
        }
final_params['loss_function']='Logloss'
final_params['eval_metric']='AUC'

In [ ]:
bayesian_final_classifier=cat.train(pool_train_redhat,final_params,iterations=5)

### Look at Catboost w GridSearch & CV Notebook

### GridSearch

In [ ]:
import catboost as cb

In [ ]:
from sklearn.model_selection import KFold
from paramsearch import paramsearch
from itertools import product,chain

In [ ]:
train_set = X_train.copy()
train_label = y_train.copy()

test_set = X_test.copy()
test_label = y_test.copy()

In [ ]:
params = {'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
#           'ctr_border_count':[50,5,10,20,100,200],
          'thread_count':4}

In [ ]:
def crossvaltest(params,train_set,train_label,cat_dims,n_splits=3):
    kf = KFold(n_splits=n_splits,shuffle=True) 
    
    res = []
    for train_index, test_index in kf.split(train_set):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)

        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    return np.mean(res)

In [ ]:
def catboost_param_tune(params,train_set,train_label,cat_dims=None,n_splits=3):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(ps.grid_search(['border_count']),
#                       ps.grid_search(['ctr_border_count']),
                      ps.grid_search(['l2_leaf_reg']),
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['depth'])):
        res = crossvaltest(prms,train_set,train_label,cat_dims,n_splits)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        print(res,prms,'best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()

In [ ]:
# train classifier with tuned parameters    
clf = cb.CatBoostClassifier(**bestparams)
clf.fit(train_set, np.ravel(train_label), cat_features=cat_dims)
res = clf.predict(test_set)

### Grid Search (internal metric)

In [ ]:
def eval_model(varname):

    train, test = df[df['is_train']==True], df[df['is_train']==False]

    # Sort the dataframe by the value you want
    train = train.sort_values(varname, ascending=False)
    test = test.sort_values(varname, ascending=False)

    # Create a unique row number on the sorted dataframe
    train['ntile'] = range(len(train))
    test['ntile'] = range(len(test))

    # Use the Pandas qcut method to divide the dataset up into n ntiles
    train['ntile'] = pd.qcut(train['ntile'], 100, labels=False)
    test['ntile'] = pd.qcut(test['ntile'], 100, labels=False)

    # Create a new dataframe containing your summary metrics; rename the columns
    joTrain = train.groupby(['ntile']).agg({'RIS_Flag_All':['count','sum'], 'RIS_All':['sum'], 'VisaTxnAmt_All':['sum'],
                                            'RIS_Flag_180D':['sum'], 'RIS_180D':['sum'], 'VisaTxnAmt_180D':['sum']
                                            }).reset_index()
    joTrain.columns = ["_".join(x) for x in joTrain.columns.ravel()]

    joTest = test.groupby(['ntile']).agg({'RIS_Flag_All':['count','sum'], 'RIS_All':['sum'], 'VisaTxnAmt_All':['sum'],
                                            'RIS_Flag_180D':['sum'], 'RIS_180D':['sum'], 'VisaTxnAmt_180D':['sum']
                                           }).reset_index()
    joTest.columns = ["_".join(x) for x in joTest.columns.ravel()] 
    
    performance=[]
    train_RIS_180D_sum = joTrain.loc[joTrain.ntile_<=9]['RIS_180D_sum'].sum()
    train_VISA_180D_sum = joTrain.loc[joTrain.ntile_<=9]['VisaTxnAmt_180D_sum'].sum()
    train_ris_pct = train_RIS_180D_sum/74289
    train_rev_pct = train_VISA_180D_sum/4486015

    test_RIS_180D_sum = joTest.loc[joTest.ntile_<=9]['RIS_180D_sum'].sum()
    test_VISA_180D_sum = joTest.loc[joTest.ntile_<=9]['VisaTxnAmt_180D_sum'].sum()
    test_ris_pct = test_RIS_180D_sum/70152
    test_rev_pct = test_VISA_180D_sum/3801526

    performance.append(('{:,.2%}'.format(train_ris_pct), '{:,.2%}'.format(train_rev_pct),'{:,.2f}'.format(train_VISA_180D_sum/train_RIS_180D_sum), 
                        '{:,.2%}'.format(test_ris_pct),  '{:,.2%}'.format(test_rev_pct), '{:,.2f}'.format(test_VISA_180D_sum/test_RIS_180D_sum )))
    
    performance_df = pd.DataFrame(performance, columns=['Train_RIS_PCT', 'Train_Rev_PCT', 'Train_RIS_Raito',
                                                        'Test_RIS_PCT', 'Test_Rev_PCT', 'Test_RIS_Raito'])

    return performance_df

In [ ]:
import catboost 
from catboost import Pool, cv, CatBoostClassifier, CatboostIpythonWidget
from sklearn.model_selection import train_test_split

def run_catboost(feature_set, iterations, depth, learning_rate, l2_leaf_reg, bagging_temperature, predict_var):
    # Create two new dataframes, one with the training rows, one with the test rows
    train, test = df[df['is_train']==True], df[df['is_train']==False]

    # Show the number of observations for the test and training dataframes
    print('Number of observations in the training data:', len(train))
    print('Number of observations in the test data:',len(test))

    X = train[feature_set]
    y = train[target]

    # CatBoost validates estimations against a subset of your training set
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234)  

    categorical_feature_indices =np.where(X_train.dtypes==np.dtype('object'))[0]
    
    result = pd.DataFrame() # create an empty dataframe before the loop
    for i in iterations: 
        for d in depth:
            for lr in learning_rate:
                for reg in l2_leaf_reg:
                    for b in bagging_temperature:
                        model=CatBoostClassifier(
                             iterations=i, 
                             depth=d, 
                             learning_rate=lr,
                             l2_leaf_reg=reg,
                             od_type='Iter',
                             od_wait=50,
                             loss_function='Logloss',
#                              metric_period=50,
                             custom_loss=['Accuracy'],
                             eval_metric='AUC',
                             thread_count=32,
                             random_seed=42)
                        model.fit(X_train, y_train, cat_features=categorical_feature_indices, eval_set=(X_test, y_test), verbose=False,plot=False)
                        df[predict_var]  = model.predict_proba(df[feature_set])[:,1]

                        performance_df = eval_model(predict_var)
                        performance_df['iteration']=i
                        performance_df['depth']=d
                        performance_df['learning_rate']=lr
                        performance_df['l2_leaf_reg']=reg
                        performance_df['bagging_temperature']=b
                #         result.append(performance_df)
                        result = pd.concat([result, performance_df], axis=0)
    
    return result

In [ ]:
iterations=[500, 600,700,800,1000]
depth=[3,4,5,6,7,8]
learning_rate=[0.005,0.01,0.03,0.05]
l2_leaf_reg=[5,10,20,30,50]
bagging_temperature=[1,5,10]

result = run_catboost(good_features, iterations, depth, learning_rate, l2_leaf_reg, bagging_temperature, 'ris_hat2')

### CatBoostRegressor

### Baseline CatBoostRegressor

In [ ]:
# updated 2019/02/05
train, validate, test = df2[df2['true_split']=='train'],df2[df2['true_split']=='val'],df2[df2['true_split']=='test']

In [ ]:
X = train[all_features]
y = train['Day180Revenue']
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1234)

X_test = test[all_features]
y_test = test['Day180Revenue']

categorical_feature_indices =np.where(X_train.dtypes==np.dtype('object'))[0]

In [ ]:
# sklearn way
# no missing data in the target label is allowed
model=cat.CatBoostRegressor(iterations=1000, 
                            learning_rate=0.03,
                            depth=6,
                            l2_leaf_reg = 60, 
                            bagging_temperature=10,
                            border_count = 10, 
                            eval_metric='RMSE', # does not support MSE/MAE
                            random_seed=123,   
                            od_type='Iter', #overfit detector wait type: iteration
                            metric_period=50,
                            od_wait=20 #overfit detector wait
                               )

model.fit(X_train, y_train,
          eval_set=(X_val,y_val),
          cat_features=categorical_feature_indices,
          use_best_model=True)

In [ ]:
import catboost as cat

from sklearn.metrics import r2_score, mean_squared_error

def rmse(y_true, y_pred):
    return round(np.sqrt(mean_squared_error(y_true, y_pred)), 5)

def run_catboost(X_train, y_train, X_val, y_val, X_test, y_test,
                 iterations,
                 learning_rate,
                 depth,
                 l2_leaf_reg,
                 bagging_temperature,
                 border_count 
                ):
    

#     categorical_feature_indices =np.where(X_train.dtypes==np.dtype('object'))[0]

    train_set=cat.Pool(data=X_train, label=y_train, cat_features=categorical_feature_indices)
    eval_set=cat.Pool(data=X_val, label=y_val, cat_features=categorical_feature_indices)
    
    model=cat.CatBoostRegressor(iterations=iterations, 
                               learning_rate=learning_rate,
                               depth=depth,
                               l2_leaf_reg = l2_leaf_reg, 
                               bagging_temperature=bagging_temperature,
                               border_count = border_count, 
                               eval_metric='RMSE', # does not support MSE/MAE
                               random_seed=123,   
                               od_type='Iter', #overfit detector wait type: iteration
                               metric_period=50,
                               od_wait=20 #overfit detector wait
                               )
    
    model.fit(train_set, 
             eval_set=eval_set,
             use_best_model=True,
             verbose=50
             )
    
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    y_pred_submit = model.predict(X_test)
    
    print(f"CatB: RMSE val: {rmse(y_val, y_pred_val)} - RMSE train: {rmse(y_train, y_pred_train)}")
#     print(model.get_best_score())
    print('test: RMSE ', rmse(y_test, y_pred_submit))
    
    print(model.get_best_iteration())
    
    return y_pred_submit, model         

In [ ]:
y_pred_submit, model   = run_catboost(X_train, y_train, X_val, y_val, X_test, y_test, 
                                      iterations=1000, 
                                      learning_rate=0.03,
                                      depth=6,
                                      l2_leaf_reg=60,
                                      bagging_temperature=10,
                                      border_count=10)

In [ ]:
feature_importances = pd.DataFrame(list(zip(X_train[all_features], model.feature_importances_)))
feature_importances.columns=['Feature Name', 'IMP']

pd.options.display.float_format='{:,.3}'.format
feature_importances.sort_values(by='IMP', ascending=False).head(30)

In [ ]:
def charting(predict_var, feature_set, df, target):
    train, validate, test = df[df['true_split']=='train'],df[df['true_split']=='val'],df[df['true_split']=='test']
    
    train[predict_var]  = model.predict(train[feature_set])
    validate[predict_var]  = model.predict(validate[feature_set])
    test[predict_var]  = model.predict(test[feature_set])

    train = train.sort_values(predict_var, ascending=False)
    validate = validate.sort_values(predict_var, ascending=False)
    test = test.sort_values(predict_var, ascending=False)

    train['ntile']=range(len(train))
    validate['ntile']=range(len(validate))
    test['ntile']=range(len(test))

    train['ntile']=pd.qcut(train['ntile'], 100, labels=False)
    validate['ntile']=pd.qcut(validate['ntile'], 100, labels=False)
    test['ntile']=pd.qcut(test['ntile'], 100, labels=False)
    
    june_train = train.groupby('ntile').agg({target:['count','mean']}).reset_index()
    june_train.columns=['ntile','count','avg_revenue_train']

    june_validate = validate.groupby('ntile').agg({target:['count','mean']}).reset_index()
    june_validate.columns=['ntile','count','avg_revenue_val']

    june_test = test.groupby('ntile').agg({target:['count','mean']}).reset_index()
    june_test.columns=['ntile','count','avg_revenue_test']
    
    train_val_test = pd.concat([june_train, june_validate['avg_revenue_val'], june_test['avg_revenue_test']], axis=1)
    return train_val_test 

In [ ]:
train_val_test = charting('rev_hat',all_features,df2,'Day180Revenue')

In [ ]:
# two group
df['RPSMax_hat'] = model.predict(df[modeling_features])
train, test = df[df['is_train']==True], df[df['is_train']==False]

print("Avg RPS in training:", train['RPSMax'].mean(), train['RPSMax_hat'].mean())
print("Avg RPS in test:", test['RPSMax'].mean(), test['RPSMax_hat'].mean())

train = train.sort_values('RPSMax_hat', ascending=False)
test = test.sort_values('RPSMax_hat', ascending=False)

# Create a unique row number on the sorted dataframe
train['ntile'] = range(len(train))
test['ntile'] = range(len(test))

# Use the Pandas qcut method to divide the dataset up into n ntiles
train['ntile'] = pd.qcut(train['ntile'], 100, labels=False)
test['ntile'] = pd.qcut(test['ntile'], 100, labels=False)

# Create a new dataframe containing your summary metrics; rename the columns
joTrain = train.groupby(['ntile']).agg({'RPSMax':['mean']}).reset_index()
joTrain.columns = ["_".join(x) for x in joTrain.columns.ravel()]

joTest = test.groupby(['ntile']).agg({'RPSMax':['mean']}).reset_index()
joTest.columns = ["_".join(x) for x in joTest.columns.ravel()]

train_test_compare = pd.merge(left=joTrain, right=joTest, on='ntile_', suffixes=('_Train', '_Test'), how='inner')
rps_result = train_test_compare[['ntile_', 'RPSMax_mean_Train', "RPSMax_mean_Test"]]
rps_result.set_index('ntile_')
rps_result[['RPSMax_mean_Train', "RPSMax_mean_Test"]].plot(figsize=(10,8))

### CatBoostRegressor & Random Search

In [ ]:
# updated 2019/02/04
categorical_feature_indices =np.where(X_train.dtypes==np.dtype('object'))[0]

In [ ]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV

def rmse(y_true, y_pred):
    return round(np.sqrt(mean_squared_error(y_true, y_pred)), 5)

def rand_cat(train_X, train_y):
    random= { "iterations"                  : sp_randint (1, 2000),
              "learning_rate"               : [0.0001, 0.001, 0.003,0.005,0.01,0.03,0.05],
              "depth"                       : np.linspace(4,16,1),
              "l2_leaf_reg"                 : np.linspace(0.1,10,100),
              "bagging_temperature"         : np.linspace(0.1,2,20),
              "od_wait"                     : [10,50,100,150],
             "cat_features": categorical_feature_indices
            }
    cat_regressor = cat.CatBoostRegressor(od_type='Iter',
                                          metric_period = 50,
                                          od_wait=20
                                         )
    
    random_search =RandomizedSearchCV(cat_regressor, 
                                      param_distributions=random, 
                                      scoring = "neg_mean_squared_error",
                                      n_iter=20, 
                                      cv = 5, 
                                      n_jobs = -1,
                                      verbose = True)


    random_search.fit(train_X,train_y)
    return random_search.best_params_, random_search.best_estimator_

In [ ]:
random_search.best_params_, random_search.best_estimator_ = rand_cat(X_train, y_train)

### CatBoostRegressor & Grid Search

In [ ]:
import catboost as cat
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from itertools import product, chain
from tqdm import tqdm

temp_prms = {'loss_function':'RMSE',
            'iterations': 500,
            'learning_rate': 0.05,
            'l2_leaf_reg':3,
            'depth':10,
            'bagging_temperature':0.8,
            'eval_metric':'RMSE'
            }

def cat_cross_cv(X, y,prms, cat_features, fold_count):
    score = []
    
    train_set=cat.Pool(data = X, label=y, cat_features=cat_features)

    rmse = cat.cv(pool = train_set, 
                params = temp_prms, 
                iterations= 500, 
                fold_count = fold_count,
                partition_random_seed = 123,
                verbose = True,
                as_pandas = True,
                metric_period = 50,
                early_stopping_rounds = 20)
    
    score.append(np.mean(rmse['test-Logloss-mean']))
    return np.mean(score)
    
score = cat_cross_cv(X_train, y_train,prms = temp_prms, cat_features = cat_col_index, fold_count = 5)

### Save & Load Catboost Model

In [ ]:
model.save_model(r'F:\Projects\June\RIS\20180808_RIS_catboost_model.dump') 

In [ ]:
model = CatBoostClassifier()
model.load_model(r'F:\Projects\June\RIS\20180816_RIS_catboost_model_w_totalbalance.dump') 

In [ ]:
# checking model version
print(catboost.__version__)

### Training on GPU

In [ ]:
#CatBoost supports training on GPUs

model = CatBoostClassifier(iterations=1000, 
                           task_type = "GPU")

### Training on Skewed Data

In [ ]:
# boxcox transformation
# log transformation

from scipy import stats
from scipy.special import boxcox1p
rps['rps_boxcox']=boxcox1p(rps['MaxRPS'],0.25)
rps['rps_boxcox'].plot(kind='hist');

https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0 


https://github.com/groverpr/Machine-Learning/tree/master/notebooks

## Catboost Feature Importance 

https://towardsdatascience.com/deep-dive-into-catboost-functionalities-for-model-interpretation-7cdef669aeed

<img src="ModelAnalysis.png">

cb = CatBoostRegressor() <br>
cb.get_feature_importance(type= "___")

 "type" possible values:
  - PredictionValuesChange
  - LossFunctionChange
  - FeatureImportance: 
      PredictionValuesChange for non-ranking metrics and LossFunctionChange for ranking metrics
  - ShapValues: 
      Calculate SHAP Values for every object
  - Interaction: 
      Calculate pairwise score between every feature

**PredictionValuesChange** <br>
Pros: It is cheap to compute as you don’t have to do multiple training or testing and you will not be storing any extra information. You will get normalized values as the ouput (all the importances will add up to 100). <br>
Cons: It may give misleading results for ranking objectives, it might put groupwise features into the top, even though they have a little influence on the resulting loss value.

**LossFunctionChange** <br>
To get this feature importance, catboost simply takes the difference between the metric (Loss function) obtained using the model in normal scenario (when we include the feature) and model without this feature (model is built approximately using the original model with this feature removed from all the trees in the ensemble). Higher the difference, the more important the feature is. It is not clearly mentioned in catboost docs how we find the model without feature. <br>

Pros & Cons: This works well for most type of problems unlike predictionvalueschange where you can get misleading results for ranking problems, at the same time, it is computationally heavy .

**SHAP Values**

SHAP stands for SHAP (SHapley Additive exPlanations) 

https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d 
    
It is the average of the marginal contributions across all permutations. It offers object-level contributions of features and overall feature importance. 

#### Pairwise Feature Importance 

With the Interaction parameter, you can find the strength of a pair of features (importance of two features together).

<img src="pairwise_feature_imp.png">

In [7]:
import pandas as pd, numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 
import catboost
from catboost import *
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
# import shap
from time import time

In [8]:
X = train[ot_features]
y = train['Sale'].values
categorical_feature_indices =np.where(X.dtypes==np.dtype('object'))[0]

from catboost import Pool
pool= Pool(X,label=y, cat_features=categorical_feature_indices)
model6.get_feature_importance(pool,type='PredictionValuesChange', prettified=True)

0.18


In [ ]:
model6.get_feature_importance(pool,type='LossFunctionChange', prettified=True)